# Text Summarization Project

Gruppo di Lavoro:
*   Pierfrancesco Lindia
*   Cristian Tedesco
*   Nabil Larhram






In [ ]:
# BLOCCO 0 — Install + GPU check
!pip -q install transformers datasets evaluate accelerate sentencepiece

import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("Torch version:", torch.__version__)


## Blocco 0 — Setup dell’ambiente e verifica GPU

In questo notebook utilizziamo Google Colab con accelerazione GPU per eseguire il fine-tuning di un modello di summarization e la successiva valutazione comparativa.

**Obiettivi di questo blocco**
- Installare le librerie necessarie per:
  - gestione dataset ( `datasets`)
  - training/inferenza del modello ( `transformers`, `accelerate`)
  - metriche di valutazione ( `evaluate`)
  - supporto al tokenizzatore (`sentencepiece`)
- Verificare che l’ambiente disponga di **CUDA** e identificare la GPU disponibile, poiché il fine-tuning su CPU sarebbe molto più lento.

**Output atteso**
Il comando stampa:
- se CUDA è disponibile (`True/False`)
- il modello di GPU assegnato (nel nostro caso **NVIDIA A100-SXM4-40GB**)
- la versione di PyTorch in uso

Questa verifica garantisce che le fasi successive (fine-tuning e generazione) possano essere eseguite in modo efficiente e riproducibile.


In [ ]:
# BLOCCO 1 — Config + seed

from transformers import set_seed
SEED = 42
set_seed(SEED)

# ===== Valutazione finale (come prima) =====
N_EVAL = 500  # numero esempi validation per confronto ROUGE

# ===== Fine-tuning (nostro) =====
N_TRAIN = 20000      # subset train (20k è ottimo per progetto)
N_VAL_FT = 1000      # validation per monitorare training

MAX_SOURCE_LEN = 512   # input articolo (training)
MAX_TARGET_LEN = 128   # lunghezza riassunto (training)

# Modello base da fine-tunare (scelta sicura)
BASE_MODEL = "facebook/bart-base"

# Cartella dove salvare il modello fine-tunato
FT_DIR = "./bart_finetuned_cnn"

print("Seed:", SEED)
print("BASE_MODEL:", BASE_MODEL)
print("N_TRAIN:", N_TRAIN, "| N_VAL_FT:", N_VAL_FT, "| N_EVAL:", N_EVAL)
print("MAX_SOURCE_LEN:", MAX_SOURCE_LEN, "| MAX_TARGET_LEN:", MAX_TARGET_LEN)
print("FT_DIR:", FT_DIR)


## Blocco 1 — Configurazione dell’esperimento e riproducibilità

In questa sezione definiamo i parametri globali dell’esperimento, in modo da rendere il workflow **riproducibile** e controllare i principali aspetti computazionali (dimensione dei subset, lunghezze massime, ecc.).

### Seed e riproducibilità
Impostiamo un seed fisso (`SEED = 42`) con `set_seed(SEED)` per rendere deterministiche (a parità di ambiente) operazioni come lo **shuffle** del dataset e alcune componenti stocastiche della generazione.

### Dimensioni dei campioni
Per mantenere il progetto computazionalmente sostenibile, utilizziamo subset controllati:
- `N_TRAIN = 20000`: numero di esempi usati per il fine-tuning (subset di *train*)
- `N_VAL_FT = 1000`: subset di *validation* usato durante il training per monitorare le performance
- `N_EVAL = 500`: subset di *validation* usato per la valutazione comparativa finale (ROUGE)

Questa separazione consente di:
1) addestrare il modello su un campione ampio ma gestibile,
2) monitorare il training su un validation set dedicato,
3) confrontare i sistemi su un evaluation set fisso e replicabile.

### Vincoli di lunghezza (token)
Impostiamo vincoli coerenti con il training di modelli seq2seq:
- `MAX_SOURCE_LEN = 512`: massimo numero di token per l’input (articolo)
- `MAX_TARGET_LEN = 128`: massimo numero di token per l’output (riassunto)

### Modello di partenza e checkpoint
- `BASE_MODEL`: checkpoint di partenza su cui eseguiamo fine-tuning.
- `FT_DIR`: directory in cui salviamo il modello fine-tunato, da riutilizzare nelle fasi di inferenza e valutazione.

L’output stampato a fine blocco riepiloga i parametri scelti, così da documentare chiaramente la configurazione dell’esperimento.


In [ ]:
# BLOCCO 2 — Load dataset + subset

from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

train_raw = dataset["train"].shuffle(seed=SEED).select(range(N_TRAIN))
val_raw_ft = dataset["validation"].shuffle(seed=SEED).select(range(N_VAL_FT))

val_eval = dataset["validation"].shuffle(seed=SEED).select(range(N_EVAL))
articles = list(val_eval["article"])
references = list(val_eval["highlights"])

print("Train subset:", len(train_raw))
print("Val (for FT) subset:", len(val_raw_ft))
print("Val (for eval) subset:", len(val_eval))

print("\nKeys:", train_raw[0].keys())
print("\nArticle preview:\n", train_raw[0]["article"][:400])
print("\nHighlights preview:\n", train_raw[0]["highlights"])


## Blocco 2 — Caricamento del dataset e costruzione dei subset

In questo blocco carichiamo il dataset **CNN/DailyMail (v3.0.0)** tramite la libreria `datasets` e costruiamo i subset necessari per le diverse fasi del workflow (fine-tuning e valutazione finale).

### Caricamento del dataset
`load_dataset("cnn_dailymail", "3.0.0")` scarica il dataset (la prima volta) e lo memorizza in cache. Il dataset è organizzato nei tre split standard:
- `train`
- `validation`
- `test`

> Nota: il warning relativo a `HF_TOKEN` indica solo che non stiamo autenticando Colab su Hugging Face. Per dataset pubblici l’autenticazione **non è necessaria** e l’avviso non influisce sull’esecuzione.

### Costruzione dei subset (riproducibili)
Per rendere l’esperimento replicabile e sostenibile:
- applichiamo `shuffle(seed=SEED)` per mescolare gli esempi in modo deterministico;
- selezioniamo poi un numero fisso di istanze con `.select(range(...))`.

Otteniamo così:
- `train_raw`: subset di train di dimensione `N_TRAIN` per il fine-tuning;
- `val_raw_ft`: subset di validation di dimensione `N_VAL_FT` per monitorare il training;
- `val_eval`: subset di validation di dimensione `N_EVAL` per la valutazione comparativa finale (ROUGE).

### Estrazione delle colonne di interesse
Dal subset di evaluation estraiamo:
- `articles`: i testi completi degli articoli (input del sistema)
- `references`: i riassunti di riferimento (*highlights*), usati come gold standard per il confronto con ROUGE

Infine stampiamo:
- le dimensioni dei subset creati,
- le chiavi disponibili in un record (`article`, `highlights`, `id`),
- un’anteprima di un articolo e del relativo riassunto di riferimento.


In [ ]:
# BLOCCO 3 — Tokenizer + preprocessing

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def preprocess(batch):
    model_inputs = tokenizer(
        batch["article"],
        max_length=MAX_SOURCE_LEN,
        truncation=True
    )
    labels = tokenizer(
        text_target=batch["highlights"],
        max_length=MAX_TARGET_LEN,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tok = train_raw.map(preprocess, batched=True, remove_columns=train_raw.column_names)
val_tok_ft = val_raw_ft.map(preprocess, batched=True, remove_columns=val_raw_ft.column_names)

print("Tokenized train example keys:", train_tok[0].keys())
print("input_ids length:", len(train_tok[0]["input_ids"]))
print("labels length:", len(train_tok[0]["labels"]))


## Blocco 3 — Tokenizzazione e preprocessing per il fine-tuning

In questo blocco prepariamo i dati nel formato richiesto da un modello **seq2seq** (encoder–decoder) per la summarization.

### Tokenizer
Carichiamo il tokenizer associato al checkpoint `BASE_MODEL`. Il tokenizer converte testo → sequenze di token/ID e gestisce il vocabolario e le regole di segmentazione coerenti con il modello.

### Preprocessing (input e target)
Definiamo una funzione `preprocess` che, per ogni batch:
- tokenizza gli **articoli** (`article`) come input del modello:
  - `max_length=MAX_SOURCE_LEN`
  - `truncation=True` per troncare gli articoli troppo lunghi al limite scelto
- tokenizza i **riassunti di riferimento** (`highlights`) come target:
  - `text_target=...` indica che stiamo tokenizzando la sequenza “label” (output atteso)
  - `max_length=MAX_TARGET_LEN`
  - `truncation=True`

Il risultato è un dizionario contenente:
- `input_ids` e `attention_mask` (input per l’encoder)
- `labels` (token del riassunto, usati per calcolare la loss in training)

### Applicazione al dataset
Applichiamo `map(..., batched=True)` ai subset `train_raw` e `val_raw_ft`, rimuovendo le colonne originali (`remove_columns=...`) per ottenere dataset già pronti per il Trainer.

### Check finale
Stampiamo:
- le chiavi disponibili nell’esempio tokenizzato,
- la lunghezza di `input_ids` e `labels`,
per verificare che i vincoli di lunghezza siano rispettati e che la struttura sia corretta.


In [ ]:
# BLOCCO 4 — Fine-tuning (compatibile)
!pip -q install rouge_score
import numpy as np
import evaluate
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)

rouge = evaluate.load("rouge")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True)

training_args = Seq2SeqTrainingArguments(
    output_dir=FT_DIR,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    seed=SEED,
    report_to="none",
    save_total_limit=2
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok_ft,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model(FT_DIR)
tokenizer.save_pretrained(FT_DIR)

print("Saved model to:", FT_DIR)



## Blocco 4 — Fine-tuning del modello (addestramento supervisionato)

In questo blocco eseguiamo il **fine-tuning supervisionato** del modello seq2seq sul task di summarization, utilizzando coppie *(articolo → highlights)*. L’obiettivo è adattare il modello al dataset scelto aggiornando i pesi tramite ottimizzazione della loss (cross-entropy sui token target).

### Caricamento del modello e data collator
- `AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)` carica l’architettura encoder–decoder.
- `DataCollatorForSeq2Seq` gestisce il padding dinamico all’interno di ogni batch e prepara correttamente le `labels` (inclusa la mascheratura con `-100` per i token di padding, così da non contribuire alla loss).

### Monitoraggio tramite ROUGE durante il training
Definiamo `compute_metrics` per calcolare ROUGE durante la valutazione intermedia:
- `predict_with_generate=True` abilita la generazione dei riassunti in fase di eval.
- Le `labels` contengono `-100` nei punti di padding; per poter decodificare correttamente le sequenze di riferimento sostituiamo `-100` con `pad_token_id`.
- Decodifichiamo predizioni e riferimenti con `batch_decode` e calcoliamo ROUGE con stemming (`use_stemmer=True`) per ridurre la sensibilità a variazioni morfologiche.

### Scelte degli iperparametri (TrainingArguments)
- `learning_rate=2e-5`: valore tipico per fine-tuning stabile di modelli transformer.
- `per_device_train_batch_size=4` e `gradient_accumulation_steps=4`: l’accumulo del gradiente simula un batch effettivo più grande (batch effettivo ≈ 16), mantenendo sotto controllo la memoria.
- `num_train_epochs=1`: impostazione iniziale conservativa, adeguata a un primo fine-tuning su subset.
- Valutazione e salvataggio “a step”:
  - `eval_strategy="steps"`, `eval_steps=500` per monitorare periodicamente
  - `save_strategy="steps"`, `save_steps=500` per salvare checkpoint intermedi
- `fp16=True` (se CUDA disponibile): abilita mixed precision per accelerare training e ridurre VRAM.

### Training e salvataggio del checkpoint
Con `Seq2SeqTrainer` avviamo l’addestramento (`trainer.train()`). Al termine salviamo:
- i pesi del modello fine-tunato (`trainer.save_model(FT_DIR)`)
- il tokenizer associato (`tokenizer.save_pretrained(FT_DIR)`)

Il checkpoint salvato verrà poi ricaricato nei blocchi successivi per la g


In [ ]:
# BLOCCO 5 — Load fine-tuned model + pipeline

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

device = 0 if torch.cuda.is_available() else -1

model_name = FT_DIR
tokenizer_ft = AutoTokenizer.from_pretrained(model_name)
model_ft = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline(
    "summarization",
    model=model_ft,
    tokenizer=tokenizer_ft,
    device=device
)

print("Loaded model from:", model_name)
print("Device:", "GPU" if device == 0 else "CPU")


## Blocco 5 — Caricamento del checkpoint fine-tunato e creazione della pipeline

Dopo il fine-tuning, ricarichiamo il modello salvato su disco e lo configuriamo per l’**inferenza** (generazione dei riassunti) tramite la `pipeline` di Hugging Face.

### Selezione del device
Impostiamo `device` in base alla disponibilità della GPU:
- `device = 0` se CUDA è disponibile (uso della GPU)
- `device = -1` altrimenti (CPU)

### Caricamento del modello fine-tunato
Carichiamo dal percorso `FT_DIR`:
- `tokenizer_ft`: il tokenizer salvato insieme al modello
- `model_ft`: i pesi del modello dopo il fine-tuning

Questo garantisce coerenza tra tokenizzazione e parametri del modello durante la generazione.

### Pipeline di summarization
Creiamo una pipeline `summarization` che incapsula:
- preprocessing (tokenizzazione)
- generazione del riassunto
- postprocessing (decodifica)

In questo modo, nei blocchi successivi possiamo richiamare il riassunto con una singola chiamata, mantenendo il codice più leggibile e modulare.

A fine blocco stampiamo:
- il percorso del checkpoint caricato
- se l’inferenza avverrà su GPU o CPU


In [ ]:
# BLOCCO 6 — Baselines + chunking + summarization

import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

MAX_INPUT_TOKENS = 1024
CHUNK_TOKENS = 900

_SENT_SPLIT = re.compile(r'(?<=[.!?])\s+')

def split_sentences(text: str):
    text = (text or "").strip()
    if not text:
        return []
    return _SENT_SPLIT.split(text)

def baseline_lead3(text: str, n_sent: int = 3) -> str:
    sents = split_sentences(text)
    if len(sents) <= n_sent:
        return (text or "").strip()
    return " ".join(sents[:n_sent]).strip()

def baseline_tfidf_extractive(text: str, n_sent: int = 3) -> str:
    sents = [s.strip() for s in split_sentences(text) if s.strip()]
    if len(sents) <= n_sent:
        return " ".join(sents).strip()

    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(sents)

    doc_vec = np.asarray(X.sum(axis=0))
    if doc_vec.ndim == 1:
        doc_vec = doc_vec.reshape(1, -1)

    sims = cosine_similarity(X, doc_vec).ravel()
    top_idx = np.argsort(-sims)[:n_sent]
    top_idx_sorted = sorted(top_idx.tolist())

    return " ".join(sents[i] for i in top_idx_sorted).strip()

def chunk_by_tokens(text: str, tok, chunk_tokens: int = 900):
    ids = tok.encode(text, add_special_tokens=False)
    return [
        tok.decode(ids[i:i + chunk_tokens], skip_special_tokens=True)
        for i in range(0, len(ids), chunk_tokens)
    ]

def _dynamic_max_len(text: str, tok, base_max: int, min_len: int, ratio: float = 0.6) -> int:
    in_len = len(tok.encode(text, add_special_tokens=False))
    cap = max(5, in_len - 1)
    proposed = int(in_len * ratio)
    lower = min(cap, min_len + 5)
    return min(base_max, cap, max(lower, proposed))

def bart_summary(text: str, max_len: int = 130, min_len: int = 30, hierarchical: bool = True) -> str:
    text = (text or "").strip()
    if not text:
        return ""

    try:
        n_tokens = len(tokenizer_ft.encode(text, add_special_tokens=False))

        if hierarchical and n_tokens > MAX_INPUT_TOKENS:
            chunks = chunk_by_tokens(text, tokenizer_ft, CHUNK_TOKENS)
            partials = []

            for ch in chunks:
                dyn_max = _dynamic_max_len(ch, tokenizer_ft, base_max=max_len, min_len=min_len, ratio=0.6)
                out = summarizer(ch, max_length=dyn_max, min_length=min_len, do_sample=False)
                partials.append(out[0]["summary_text"])

            merged = " ".join(partials)

            final_min = 40
            final_base_max = 120
            dyn_final_max = _dynamic_max_len(merged, tokenizer_ft, base_max=final_base_max, min_len=final_min, ratio=0.7)

            out_final = summarizer(merged, max_length=dyn_final_max, min_length=final_min, do_sample=False)
            return out_final[0]["summary_text"].strip()

        dyn_max = _dynamic_max_len(text, tokenizer_ft, base_max=max_len, min_len=min_len, ratio=0.6)
        out = summarizer(text, max_length=dyn_max, min_length=min_len, do_sample=False)
        return out[0]["summary_text"].strip()

    except Exception as e:
        print(f"[BART] Error: {e}")
        return "Error in summary generation."


## Blocco 6 — Baseline estrattive, gestione di input lunghi e funzione di generazione

In questo blocco definiamo:
1) due **baseline estrattive** (non neurali) per il confronto,
2) una strategia robusta per gestire articoli lunghi,
3) una funzione unificata per generare riassunti con il modello fine-tunato.

---

### 1) Baseline estrattiva Lead-3
`baseline_lead3` implementa una strategia semplice e molto usata nel dominio news:
- segmenta l’articolo in frasi,
- restituisce la concatenazione delle **prime 3 frasi**.

È una baseline forte perché molti articoli giornalistici seguono una struttura a “piramide invertita”, in cui le informazioni principali compaiono all’inizio.

---

### 2) Baseline estrattiva TF-IDF + similarità coseno
`baseline_tfidf_extractive`:
- divide l’articolo in frasi,
- costruisce una rappresentazione TF-IDF delle frasi,
- calcola un vettore “documento” aggregato (somma delle feature),
- seleziona le frasi con maggiore similarità coseno rispetto al vettore documento.

Questo approccio è interpretabile e completamente estrattivo, ma può produrre output meno coesi (frasi non contigue o ridondanti).

---

### 3) Gestione degli input lunghi: chunking token-based
I modelli transformer hanno un limite massimo di lunghezza in input; per articoli molto lunghi utilizziamo una strategia di **chunking basata sui token**:
- `chunk_by_tokens` spezza l’articolo in segmenti da `CHUNK_TOKENS` token,
- questo evita errori di overflow e garantisce compatibilità con il modello.

---

### 4) Controllo dinamico della lunghezza dell’output
La funzione `_dynamic_max_len` imposta un `max_length` proporzionale alla lunghezza dell’input:
- evita riassunti troppo lunghi rispetto al testo,
- mantiene coerenza tra `min_length` e `max_length`,
- riduce generazioni “sproporzionate” su input molto corti o molto lunghi.

---

### 5) Funzione `bart_summary`: modalità standard e gerarchica
`bart_summary` genera riassunti con due modalità:
- **Standard**: se l’articolo è entro `MAX_INPUT_TOKENS`, genera direttamente un riassunto.
- **Gerarchica**: se l’articolo supera il limite:
  1) genera un riassunto per ciascun chunk,
  2) concatena i riassunti parziali,
  3) genera un **riassunto finale** sul testo aggregato.

In questo modo otteniamo un comportamento robusto anche su articoli lunghi, mantenendo la pipeline stabile in inferenza e comparabile con le baseline.


In [ ]:
# BLOCCO 7 — Generate summaries + ROUGE

from tqdm.auto import tqdm
import pandas as pd
import evaluate

pred_bart, pred_lead3, pred_tfidf = [], [], []

for art in tqdm(articles, desc="Generating summaries"):
    pred_bart.append(bart_summary(art, hierarchical=True))
    pred_lead3.append(baseline_lead3(art, n_sent=3))
    pred_tfidf.append(baseline_tfidf_extractive(art, n_sent=3))

rouge = evaluate.load("rouge")

def rouge_scores(preds, refs):
    return rouge.compute(predictions=preds, references=refs, use_stemmer=True)

scores_bart = rouge_scores(pred_bart, references)
scores_lead3 = rouge_scores(pred_lead3, references)
scores_tfidf = rouge_scores(pred_tfidf, references)

df = pd.DataFrame([
    {"system": f"BART fine-tuned ({BASE_MODEL})", **scores_bart},
    {"system": "Baseline Lead-3", **scores_lead3},
    {"system": "Baseline TF-IDF extractive", **scores_tfidf},
])

cols = ["system", "rouge1", "rouge2", "rougeL", "rougeLsum"]
print(df[cols].to_string(index=False))


## Blocco 7 — Generazione dei riassunti e valutazione quantitativa (ROUGE)

In questo blocco eseguiamo la **valutazione comparativa finale** sui `N_EVAL` articoli dello split *validation* (subset fisso e riproducibile definito nel Blocco 2).

### 1) Generazione delle predizioni
Per ogni articolo in `articles` generiamo tre riassunti:
- `pred_bart`: riassunto prodotto dal **modello fine-tunato** tramite `bart_summary` (con gestione gerarchica degli input lunghi)
- `pred_lead3`: baseline estrattiva **Lead-3**
- `pred_tfidf`: baseline estrattiva **TF-IDF + coseno**

Le tre liste di predizioni hanno la stessa cardinalità e sono confrontabili una-a-una con i riferimenti `references` (highlights).

### 2) Valutazione con ROUGE
Utilizziamo la metrica **ROUGE** per confrontare automaticamente i riassunti generati con i riassunti di riferimento:
- **ROUGE-1**: sovrapposizione di unigrammi
- **ROUGE-2**: sovrapposizione di bigrammi
- **ROUGE-L** e **ROUGE-Lsum**: basate sulla *Longest Common Subsequence*, utili per catturare similarità di struttura e ordine

Impostiamo `use_stemmer=True` per ridurre la sensibilità a variazioni morfologiche (es. plurali/tempi verbali).

### 3) Tabella riassuntiva dei risultati
Raccogliamo i punteggi ROUGE in un `DataFrame` e stampiamo una tabella comparativa con le quattro metriche principali, così da osservare immediatamente le differenze tra:
- approccio neurale (modello fine-tunato)
- approcci estrattivi (Lead-3 e TF-IDF)

Questa sezione costituisce la base quantitativa su cui poggia l’interpretazione dei risultati.


In [ ]:
# BLOCCO 8 — Qualitative check (first 5)

import textwrap
import re

_SENT_SPLIT_READABLE = re.compile(r'(?<=[.!?])\s+')

def pretty_wrap(text: str, width: int = 90) -> str:
    text = (text or "").strip()
    if not text:
        return ""
    return "\n".join(textwrap.wrap(text, width=width))

W = 90

print("\n--- Qualitative examples (first 5) ---\n")
for i in range(5):
    print(f"Example {i+1}\n")
    print("REFERENCE:\n" + pretty_wrap(references[i], width=W) + "\n")
    print("BART (fine-tuned):\n" + pretty_wrap(pred_bart[i], width=W) + "\n")
    print("LEAD-3:\n" + pretty_wrap(pred_lead3[i], width=W) + "\n")
    print("TF-IDF:\n" + pretty_wrap(pred_tfidf[i], width=W))
    print("-" * 90)


## Blocco 8 — Valutazione qualitativa (ispezione manuale di esempi)

Le metriche automatiche (es. ROUGE) misurano principalmente la **sovrapposizione lessicale** tra riassunto generato e riferimento, ma non catturano completamente aspetti come:
- scorrevolezza e coerenza discorsiva,
- ridondanza,
- completezza informativa,
- eventuali dettagli aggiunti o omessi.

Per questo motivo, affianchiamo alla valutazione quantitativa una breve **ispezione qualitativa**.

### Procedura
Per i primi 5 esempi del subset di evaluation (`val_eval`):
- stampiamo il riassunto di riferimento (`REFERENCE`, ovvero gli *highlights*),
- confrontiamo i tre output:
  - **BART (fine-tuned)**
  - **Lead-3**
  - **TF-IDF**

### Formattazione dell’output
La funzione `pretty_wrap` va a capo automaticamente a una larghezza fissa (`W = 90`), migliorando la leggibilità in console/notebook e facilitando il confronto visivo tra i diversi metodi.

### Scopo dell’analisi qualitativa
Questa sezione permette di osservare concretamente i diversi trade-off:
- Lead-3: forte aderenza all’incipit dell’articolo (approccio posizionale)
- TF-IDF: selezione di frasi lessicalmente salienti (approccio basato su similarità)
- Modello fine-tunato: maggiore (potenziale) capacità di sintesi e riformulazione

Le osservazioni qualitative vengono poi utilizzate per interpretare e contestualizzare i risultati ROUGE ottenuti nel blocco precedente.
